In [1]:
import logging

from chatbot.conversation_engine import ConversationEngine
# 기능 테스트를 위해 diary_manager를 별도로 import
from chatbot.chatbot_modules.diary_manager import DiaryManager

logging.basicConfig(level=logging.INFO, format='%(message)s')
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("openai").setLevel(logging.WARNING)

c:\Users\bsjun\anaconda3\envs\vectordb\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def run_test():
    print("=============================================")
    print("• 모드 전환: '/토글' (감성 <-> 정보)")
    print("• 대화 종료: '그만', '종료'")
    print(" [다이어리 관리 명령어]")
    print(" • 생성 트리거: '다이어리' 입력")
    print(" • 목록 조회:   '/목록'")
    print(" • 내용 읽기:   '/읽기 2025-11-28'")
    print(" • 기록 삭제:   '/삭제 2025-11-28'")
    print("=============================================\n")

    # 1. 엔진 및 매니저 초기화
    try:
        engine = ConversationEngine()
        # 관리 기능(목록, 삭제)을 위해 DiaryManager 별도 인스턴스 사용 (데이터는 공유됨)
        diary_admin = DiaryManager() 
    except Exception as e:
        print(f"❌ [오류] 엔진 초기화 실패: {e}")
        return

    # 테스트용 사용자 ID 설정
    user_id = "test_user_01"
    current_mode = "chat"

    # 2. 환영 인사
    welcome_msg = engine.session_manager.get_welcome_message(user_id)
    print(f"[System] 현재 모드: {current_mode.upper()}")
    print(f"[AI]: {welcome_msg}\n")

    # 3. 메인 루프
    while True:
        try:
            mode_indicator = "💬감성" if current_mode == "chat" else "ℹ️정보"
            
            user_input = input(f"[{mode_indicator} | 나]: ").strip()

            if not user_input:
                continue
                
            # ===============================================================
            # [기능 1] 다이어리 관리 명령어 처리
            # ===============================================================
            
            # 1-1. 목록 조회
            if user_input == "/목록":
                print("\n📖 [다이어리 목록]")
                diaries = diary_admin.list_diaries(user_id)
                if diaries:
                    for d in diaries:
                        print(f" - {d['date']} {d['emoji']} {d['tags']}")
                else:
                    print("   (저장된 다이어리가 없습니다)")
                print("")
                continue

            # 1-2. 내용 읽기
            if user_input.startswith("/읽기"):
                parts = user_input.split()
                if len(parts) < 2:
                    print("⚠️ 날짜를 입력해주세요. 예: /읽기 2025-11-28")
                    continue
                
                target_date = parts[1]
                content = engine.session_manager.get_diary_entry(user_id, target_date)
                if content:
                    print(f"\n📄 [{target_date} 다이어리 내용]\n" + "-"*30)
                    print(content)
                    print("-"*30 + "\n")
                else:
                    print(f"❌ {target_date}에 작성된 다이어리가 없습니다.\n")
                continue

            # 1-3. 기록 삭제
            if user_input.startswith("/삭제"):
                parts = user_input.split()
                if len(parts) < 2:
                    print("⚠️ 날짜를 입력해주세요. 예: /삭제 2025-11-28")
                    continue
                
                target_date = parts[1]
                result_msg = diary_admin.delete_diary(user_id, target_date)
                print(f"\n🗑️ {result_msg}\n")
                continue

            # ===============================================================
            # [기능 2] 일반 대화 및 엔진 제어
            # ===============================================================

            # 종료
            if user_input in ["그만", "종료", "exit", "quit"]:
                engine.session_manager.update_last_visit(user_id)
                print(">>> 시스템을 종료합니다.")
                break

            # 모드 토글
            if user_input in ["/토글", "/toggle"]:
                current_mode = "info" if current_mode == "chat" else "chat"
                print(f"\n>>> 🔄 모드를 [{current_mode.upper()}]로 전환했습니다.\n")
                continue

            # 챗봇 엔진 호출 (다이어리 생성 트리거 '다이어리' 입력 시 내부 로직 동작)
            response = engine.process_user_message(user_id, user_input, mode=current_mode)
            print("[나]", user_input)
            print(f"[{mode_indicator} | AI]: {response}\n")

        except KeyboardInterrupt:
            print("\n\n>>> 테스트를 강제 중단합니다.")
            break
        except Exception as e:
            print(f"\n❌ [오류] 실행 중 문제가 발생했습니다: {e}\n")

In [5]:
run_test()

• 모드 전환: '/토글' (감성 <-> 정보)
• 대화 종료: '그만', '종료'
 [다이어리 관리 명령어]
 • 생성 트리거: '다이어리' 입력
 • 목록 조회:   '/목록'
 • 내용 읽기:   '/읽기 2025-11-28'
 • 기록 삭제:   '/삭제 2025-11-28'

[System] 현재 모드: CHAT
[AI]: 안녕하세요, 회원님. 오늘은 좀 어떠신가요?



INFO:chatbot.chatbot_modules.empathy_agent:>>> [Agent Active] Empathy Agent (감성 모드)
INFO:chatbot.chatbot_modules.empathy_agent:⚖️ 진지함 점수: 0.0 -> 0.09 (입력무게: 3)


[나] 오늘은 좀 괜찮아
[💬감성 | AI]: 오늘이 조금 더 나은 날이라니 정말 다행이네요. 혹시 오늘 특별히 기분이 좋아질 만한 일이 있으셨나요? 



INFO:chatbot.chatbot_modules.empathy_agent:>>> [Agent Active] Empathy Agent (감성 모드)
INFO:chatbot.chatbot_modules.empathy_agent:⚖️ 진지함 점수: 0.09 -> 0.09 (입력무게: 1)


[나] 오늘 날씨가 좀 포근했어
[💬감성 | AI]: 포근한 날씨가 마음을 조금 더 편안하게 해주었나 봐요. 그런 날에는 산책이나 가벼운 외출도 좋을 것 같은데, 혹시 오늘 특별히 기억에 남는 순간이 있으셨나요?



...

[Tool: 활동 추천]


INFO:chatbot.chatbot_modules.empathy_agent:>>> [Agent Active] Empathy Agent (감성 모드)
INFO:chatbot.chatbot_modules.empathy_agent:⚖️ 진지함 점수: 0.3 -> 0.21 (입력무게: 0)


[Tool] 검색 결과
 ['- 베개 베고 쉬기 (기대효과: 평온/이완)', '- 그냥 눈 감고 쉬기 (기대효과: 평온/이완)', '- 소설·시 읽기 (기대효과: 평온/이완)']
[나] 마음이 무거울때 뭘하면 좋을까?
[💬감성 | AI]: 마음이 무거울 때는 잠시 쉬어가는 것도 좋은 방법이에요. 혹시 괜찮으시다면, 베개에 기대어 잠시 눈을 감고 쉬어보셔도 좋을 것 같아요. 또는 소설이나 시를 읽으며 잠시 다른 세계에 빠져보는 것도 마음의 짐을 덜어줄 수 있을 거예요. 언제든지 부담 없이 시도해보세요.

[나] 다이어리
[💬감성 | AI]: 오늘 나눈 대화로 다이어리를 생성할까요? (Y/N)



INFO:chatbot.chatbot_modules.diary_manager:다이어리 생성 완료: test_user_01, 2025-11-30


[나] y
[💬감성 | AI]: [2025/11/30] 🌤️ #포근한 날씨 #주토피아 #생각

사용자님은 오늘 포근한 날씨 속에서 마음의 여유를 조금 되찾으셨던 것 같아요.

영화 '주토피아'의 OST가 나오던 장면에서 즐거움을 느끼셨다고 말씀하셨습니다.

삶에 대한 깊은 생각 속에서도 오늘의 따뜻함이 내일의 밝은 시작이 되길 바랍니다.


📖 [다이어리 목록]
 - 2025-11-30 🌤️ #포근한 #주토피아 #생각

⚠️ 날짜를 입력해주세요. 예: /읽기 2025-11-28

📄 [2025-11-30 다이어리 내용]
------------------------------
[2025/11/30] 🌤️ #포근한 날씨 #주토피아 #생각

사용자님은 오늘 포근한 날씨 속에서 마음의 여유를 조금 되찾으셨던 것 같아요.

영화 '주토피아'의 OST가 나오던 장면에서 즐거움을 느끼셨다고 말씀하셨습니다.

삶에 대한 깊은 생각 속에서도 오늘의 따뜻함이 내일의 밝은 시작이 되길 바랍니다.
------------------------------

>>> 시스템을 종료합니다.
